# Data Wrangling

## Dataset - WeRateDogs&trade; Twitter Archive

***By Kartik Nanduri***<br>
**Dated: 21st Nov, 2018.**

In [ ]:
# importing all the necessary libraries
import os
import pandas as pd
import requests as req
import re

1. [x] **The file given at hand `twitter-archive-enhanced.csv`**

In [ ]:
# all the requried files for this project are in the list files_list
files_list = ['twitter-archive-enhanced.csv', 'image-predictions.tsv', 'tweet_json.txt']

In [ ]:
# reading the twitter archive file
archive = pd.read_csv(files_list[0])

# taking at random file entries for the archive file
archive.sample(5)

2. [x] **Fetching the data from url and saving it to local drive - `image-predictions.tsv`**

In [ ]:
# reading the file from internet using the requests library
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
res = req.get(url)

with open(url.split('/')[-1], mode = "wb") as op_file:
    op_file.write(res.content)

In [ ]:
# checking if fetched the data right way
img_pre_test = pd.read_csv(files_list[1], delimiter = "\t", encoding = 'utf-8')
img_pre_test.sample(5)

# we did it the right way, Yay! it worked.

3. [x] **Getting data from Twitter&trade;**

In [ ]:
# importing all the necessary libraries for accessing Twitter via API
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

In [ ]:
# setting up all the necessary placeholders for API
consumer_key = 'x'
consumer_secret = 'x'
access_token = 'x'
access_secret = 'x'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth_handler = auth,
                 wait_on_rate_limit = True, 
                 wait_on_rate_limit_notify = True)

In [ ]:
def fetch_and_save(ids, api_ins, one_id = None):
    '''
    This function will fetch data with associated id in ids list
    ids (List Object): a list all tweets
    api_ins (Tweepy Object): api object instance, will be used to query twitter for data
    one_id (int): use when you want to query only for one tweet
    failed_ids (List Object): a list will be retured so that, this fuction can be called once again on those ids
    '''
    new_file_name = ''; failed_ids = []
    
    # checking if file exists
    if os.path.exists(files_list[2]):
        temp = [s for s in os.listdir() if "tweet_json" in s]
        new_file_name = files_list[2].split('.')[0] + "_" + str(len(temp)) + ".txt"
    else:
        new_file_name = files_list[2]
    
    # querying a list of ids
    if one_id == None:
        with open(new_file_name, mode = 'w') as outfile:
            for one_id in ids:
                try:
                    page = api_ins.get_status(one_id, tweet_mode='extended')
                    json.dump(page._json, outfile)
                    outfile.write('\n')
                except Exception as e:
                    print("Error for: " + str(one_id) + " - " + str(e))
                    failed_ids.append(one_id)
    
    # querying a single id
    else:
        with open(new_file_name, mode = 'w') as outfile:
            try:
                page = api_ins.get_status(one_id, tweet_mode='extended')
                json.dump(page._json, outfile)
                outfile.write('\n')
            except Exception as e:
                print("Error for: " + str(one_id) + " - " + str(e))
                failed_ids.append(one_id)
    
    return failed_ids

In [ ]:
# starting the timer
start = timer()

# passing the list of ids to the fuction fetch_and_save()
tweet_ids = archive['tweet_id'].tolist()

# fetching data 1st iteration
test_one = fetch_and_save(tweet_ids, api)

# ending the timer
end = timer()

# calculating the runtime for fetch_and_save
print("That took about {} mins.".format(round((end - start)/60, 1)))

In [ ]:
# no of erroneous ids
len(test_one)

In [ ]:
# checking if test_one has duplicate ids
assert len(test_one) == len(set(test_one))

# we can see that there are no duplicates

In [ ]:
# passing this failed list to the fetch_and_save function

# starting the timer
start = timer()

# fetching data 2nd iteration
test_two = fetch_and_save(test_one, api)

# ending the timer
end = timer()

# calculating the runtime for fetch_and_save
print("That took about {} secs.".format(round(end - start, 1)))